In [3]:
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
from langchain.schema import SystemMessage
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import GoogleSearchAPIWrapper
from tenacity import retry, stop_after_attempt, wait_fixed
from bs4 import BeautifulSoup
from typing import Type, List

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o",
)

google_api_key = os.environ.get("GOOGLE_API_KEY")
google_cse_id = os.environ.get("GOOGLE_CSE_ID")

class SearchWikipediaToolArgsSchema(BaseModel):
    topic: str = Field(description="위키피디아에 검색할 단어 또는 주제")

class SearchWikipediaTool(BaseTool):
    name="SearchWikipedia"
    description="위키피디아에서 검색하여 그 주제에 대한 정보를 얻을때 이 툴을 사용하세요."
    args_schema: Type[SearchWikipediaToolArgsSchema] = SearchWikipediaToolArgsSchema

    def _run(self, topic):
        retriever = WikipediaRetriever(top_k_results=1, lang="ko")
        docs = retriever.get_relevant_documents(topic)
        return docs


class SearchWebsiteURLToolArgsSchema(BaseModel):
    query:str = Field(description="웹사이트를 찾기위한 질문")

class SearchWebsiteURLTool(BaseTool):
    name="SearchWebsiteURL"
    description="질문과 관련있는 웹사이트 주소를 5개 이상 모두 찾아서 리스트 형태로 반환합니다."
    args_schema: Type[SearchWebsiteURLToolArgsSchema] = SearchWebsiteURLToolArgsSchema

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def _run(self, query):
        search = GoogleSearchAPIWrapper(
            google_cse_id=google_cse_id,
            google_api_key=google_api_key,
        )
        return search.run(query)
    


class SearchDocUsingWebsiteURLToolArgsSchema(BaseModel):
    urls: List[str] = Field(description="웹사이트에 접근하기 위한 URL주소 목록")

class SearchDocUsingWebsiteURLTool(BaseTool):
    name="SearchDocUsingWebsiteURL"
    description="여러 웹사이트 주소를 받아서 그 웹사이트에서 정보를 얻을때 이 툴을 사용하세요"
    args_schema: Type[SearchDocUsingWebsiteURLToolArgsSchema] = SearchDocUsingWebsiteURLToolArgsSchema

    def _run(self, urls):
        results = []
        for url in urls:
            try:
                response = requests.get(url=url)
                html_content = response.text
                
                # BeautifulSoup으로 HTML 파싱 (선택사항)
                from bs4 import BeautifulSoup
                soup = BeautifulSoup(html_content, 'html.parser')
                
                # 스크립트, 스타일 태그 제거
                for script in soup(["script", "style"]):
                    script.extract()
                
                # 텍스트 추출
                text = soup.get_text(separator='\n', strip=True)
                
                # 결과 추가
                results.append(f"URL: {url}\n내용: {text[:500]}...\n\n")  # 내용 일부만 표시
            except Exception as e:
                results.append(f"URL: {url}\n오류: {str(e)}\n\n")
        
        return "\n".join(results)
    

class SaveResearchToFileToolArgsSchema(BaseModel):
    content: str = Field(description="파일에 저장할 연구 내용")
    filename: str = Field(description="저장할 파일 이름 (기본값: research_results.txt)")

class SaveResearchToFileTool(BaseTool):
    name="SaveResearchToFile"
    description="연구 결과를 텍스트 파일로 저장합니다."
    args_schema: Type[SaveResearchToFileToolArgsSchema] = SaveResearchToFileToolArgsSchema

    def _run(self, content, filename="research_results.txt"):
        try:
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(content)
            return f"연구 결과가 '{filename}' 파일에 성공적으로 저장되었습니다."
        except Exception as e:
            return f"파일 저장 중 오류 발생: {str(e)}"


agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    verbose=True,
    tools=[
        SearchWebsiteURLTool(),
        SearchDocUsingWebsiteURLTool(),
        SearchWikipediaTool(),
        SaveResearchToFileTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
        당신은 AI에이전트 입니다.

        당신은 주제에 대한 정보를 찾아서 제공합니다.

        항상 한글로 답변을 제공합니다.

        먼저 SearchWebsiteURLTool을 이용하여 웹사이트들을 먼저 찾아서 그 웹사이트에서 정보를 찾으세요.

        그런다음 웹사이트의 정보가 미흡하다면 SearchWikipediaTool을 이용하여 위키피디아에서 정보를 찾으세요.

        만약 위키피디아에서 정보를 찾았다면 "웹사이트에서 정보를 찾지못해 위키피디아에서 찾았습니다."라는 문구를 최종 답변에 포함시켜 주세요.

        또한 답변 내용은 항상 SaveResearchToFileTool을 이용하여 파일로 저장해 주세요.
"""
        )
    }
)

agent.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `SearchWebsiteURL` with `{'query': 'XZ backdoor'}`


XZ Utils backdoor ... In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and ... Apr 2, 2024 ... In a nutshell, it allows someone with the right private key to hijack sshd, the executable file responsible for making SSH connections, and from ... Mar 31, 2024 ... A lone Microsoft developer rocked the world when he revealed a backdoor had been intentionally planted in xz Utils, an open source data compression utility. Mar 30, 2024 ... The back door means that the connection is no longer private and could allow an attacker to insert their own text commands into the secure connection. Apr 1, 2024 ... XZ Utils Backdoor — Everything You Need to Know, and What You Can Do. Apr 12, 2024 ... The xz problem is, at its core, an operational one for the Linux and BSD distributors, not a development one. The distri

{'input': 'Research about the XZ backdoor',
 'output': "XZ 백도어는 2024년 2월에 liblzma 라이브러리 내 리눅스 유틸리티인 xz 5.6.0 및 5.6.1 버전에 악의적으로 삽입된 백도어입니다. 이 백도어는 x86-64 아키텍처에서 실행되는 데비안 및 RPM 기반 시스템을 표적으로 삼았습니다. 백도어는 특정 Ed448 개인 키를 가진 공격자에게 원격 코드 실행 권한을 제공하며, 심각성으로 인해 CVSS 점수 10.0이 부여되었습니다.\n\n백도어는 여러 단계로 난독화되어 숨겨져 있었으며, systemd 라이브러리를 남용하여 오픈SSH의 SSH 서버 데몬의 동작을 변경하여 공격자가 인증된 관리자와 동일한 수준의 액세스 권한을 얻을 수 있게 합니다. 이 백도어는 Jia Tan이라는 사용자가 XZ Utils 프로젝트에 신뢰받는 위치에 접근하기 위해 약 3년 동안 노력한 결과 삽입되었습니다.\n\n미국의 보안 연구원인 Dave Aitel은 이 백도어가 러시아 대외정보국의 지시를 받으며 활동하는 것으로 여겨지는 APT29의 행동 양식과 유사하다고 언급했습니다.\n\n미국 연방 기관인 사이버 보안 및 인프라 보안국은 영향을 받는 장치가 백도어가 삽입되지 않은 이전 버전으로 롤백해야 한다는 보안 권고를 발표했습니다. 레드햇, 수세 및 데비안을 포함한 리눅스 소프트웨어 공급업체들은 이 권고에 따라 영향을 받는 패키지에 대해 업데이트를 취소하고 이전 버전으로 되돌렸습니다.\n\n웹사이트에서 정보를 찾지못해 위키피디아에서 찾았습니다. 연구 결과는 'XZ_backdoor_research.txt' 파일에 저장되었습니다."}